In [1]:
print("transformer")

transformer


词嵌入，embeddings，让文本变为数字，embeddings越好模型就会越好。
相似的单词会赋予相似的数据，这样距离比较近。

attention计算出一个句子中每个单词之间的相关性。
三种表示相关性的方法
1. 点积（向量之间，相同坐标轴的值相乘然后求和）
2. 余弦相似度（计算两个向量之间的夹角，可以用点积/模长）
3. 比例点积（点积/sqrt(维度)，利用维度大小对点积进行缩放）

In [2]:
import numpy as np

text1 = '我喜欢的水果是橙子和苹果'
text2 = '相比苹果我更加喜欢国产的华为'

In [3]:
import spacy

nlp = spacy.load('zh_core_web_sm')

In [4]:
doc = nlp(text1)
doc

我喜欢的水果是橙子和苹果

In [5]:
emd_dim = 10

In [6]:
dics = {}
for token in doc:
    dics[token.text] = token.vector[:emd_dim]
dics

{'我': array([ 1.4686487 , -0.72315717, -1.2931682 ,  0.24932906,  2.3291414 ,
         1.7527952 , -0.25596786,  0.8271904 , -0.51447475, -1.0562801 ],
       dtype=float32),
 '喜欢': array([ 0.51233214,  0.6927922 ,  0.00741255,  1.460608  , -0.0323258 ,
        -0.02877208, -0.2949775 ,  0.31405067, -0.70030355,  0.7001613 ],
       dtype=float32),
 '的': array([ 3.3848634 ,  2.5763922 , -1.3831755 ,  1.7510111 , -2.0771494 ,
         2.153421  , -0.79614574, -0.2623603 , -0.19018924,  0.55982596],
       dtype=float32),
 '水果': array([ 0.93435514, -2.026781  , -0.7905892 , -1.1106956 ,  1.0638433 ,
        -0.62800986,  3.8172576 , -1.0496758 , -0.55736905, -0.9285867 ],
       dtype=float32),
 '是': array([ 0.66781914,  0.3840046 ,  0.05266917, -0.38295984, -1.6400721 ,
        -1.5146024 ,  1.938287  , -1.456138  ,  0.11141145, -2.9090867 ],
       dtype=float32),
 '橙子': array([ 1.0052307 , -1.7132874 , -2.3313982 , -0.06937416,  1.9805148 ,
        -2.4350555 ,  0.4545729 , -2.1072867

In [7]:
import pandas as pd
X = pd.DataFrame(dics)
X.T

,0,1,2,3,4,5,6,7,8,9
我,1.468649,-0.723157,-1.293168,0.249329,2.329141,1.752795,-0.255968,0.827190,-0.514475,-1.056280
喜欢,0.512332,0.692792,0.007413,1.460608,-0.032326,-0.028772,-0.294977,0.314051,-0.700304,0.700161
的,3.384863,2.576392,-1.383175,1.751011,-2.077149,2.153421,-0.796146,-0.262360,-0.190189,0.559826
水果,0.934355,-2.026781,-0.790589,-1.110696,1.063843,-0.628010,3.817258,-1.049676,-0.557369,-0.928587
是,0.667819,0.384005,0.052669,-0.382960,-1.640072,-1.514602,1.938287,-1.456138,0.111411,-2.909087
橙子,1.005231,-1.713287,-2.331398,-0.069374,1.980515,-2.435055,0.454573,-2.107287,-0.501081,-1.240167
和,0.915805,-1.751494,1.843134,-0.005472,-0.945925,-0.718926,1.925642,3.552628,2.092016,-0.643384
苹果,-1.193919,-0.804299,0.572513,-1.500060,0.772428,-1.325299,3.144667,-0.644317,-0.995969,0.586208


### 初始化q,k,v三个矩阵

In [8]:
d_k = 6
Wq = np.random.randn(emd_dim, d_k)
Wq

array([[-0.30194687, -1.95187202, -0.43896892, -0.36676987, -0.98059633,
         1.77312025],
       [-1.089866  ,  1.84046523,  0.74834629, -0.73706806,  0.4013879 ,
        -0.01255492],
       [-0.10439059, -0.21302421,  1.20830578,  0.94744305, -1.57203915,
         0.44527478],
       [ 0.74270187, -0.52083366,  0.09618718,  1.18775936,  0.84039451,
         0.10267333],
       [ 0.46067398, -0.46215466, -0.07424021, -0.42661075,  0.08835355,
        -0.01306486],
       [-0.5851177 , -1.11123371, -1.19496385, -0.74397331, -0.63137786,
        -1.20007764],
       [-0.37295409, -0.80114679,  0.29352487, -1.66218431, -0.1037065 ,
        -1.18824255],
       [ 1.43463115, -0.52172385, -1.64677092, -0.34912491, -1.09040001,
        -0.38344248],
       [-2.01055494,  0.12878241, -0.46412083,  0.22925222,  0.56231776,
         2.00603732],
       [ 0.5756863 ,  0.18523678, -0.89878697, -0.55923667, -1.5663055 ,
         0.02740595]])

In [9]:
Wk = np.random.randn(emd_dim, d_k)
Wv = np.random.randn(emd_dim, d_k)

### 计算得到q,k,v矩阵

In [10]:
X.T.shape

(8, 10)

In [11]:
Wq

array([[-0.30194687, -1.95187202, -0.43896892, -0.36676987, -0.98059633,
         1.77312025],
       [-1.089866  ,  1.84046523,  0.74834629, -0.73706806,  0.4013879 ,
        -0.01255492],
       [-0.10439059, -0.21302421,  1.20830578,  0.94744305, -1.57203915,
         0.44527478],
       [ 0.74270187, -0.52083366,  0.09618718,  1.18775936,  0.84039451,
         0.10267333],
       [ 0.46067398, -0.46215466, -0.07424021, -0.42661075,  0.08835355,
        -0.01306486],
       [-0.5851177 , -1.11123371, -1.19496385, -0.74397331, -0.63137786,
        -1.20007764],
       [-0.37295409, -0.80114679,  0.29352487, -1.66218431, -0.1037065 ,
        -1.18824255],
       [ 1.43463115, -0.52172385, -1.64677092, -0.34912491, -1.09040001,
        -0.38344248],
       [-2.01055494,  0.12878241, -0.46412083,  0.22925222,  0.56231776,
         2.00603732],
       [ 0.5756863 ,  0.18523678, -0.89878697, -0.55923667, -1.5663055 ,
         0.02740595]])

In [12]:
np.dot(X.T, Wq)     # 使用的数值

array([[ 2.42071673, -7.56454575, -5.24104308, -2.62293083,  0.10087579,
        -1.14499806],
       [ 2.54785014, -0.32836632, -0.42824267,  0.90708906, -0.7954779 ,
        -0.06761633],
       [-3.97678917, -3.06146867, -3.69627729, -2.02883487, -0.79741368,
         3.65660792],
       [-0.3015599 , -7.3551844 ,  1.62479786, -6.49071561,  0.959894  ,
        -3.32102613],
       [-5.49003371,  0.71485989,  7.48247767, -0.16744735,  5.91194577,
         1.40146211],
       [ 1.19353518, -2.35095205,  3.16651255,  0.12835688,  7.55711189,
         2.88375437],
       [ 1.22282809, -7.41079957, -4.23469847,  0.03156975, -6.02217767,
         3.86975756],
       [ 1.43725685,  0.42229141,  3.91547662, -5.11033616, -1.50951862,
        -5.89703725]])

In [13]:
Q = X.T @ Wq    # 原有的索引不变
Q

,0,1,2,3,4,5
我,2.420717,-7.564546,-5.241043,-2.622931,0.100876,-1.144998
喜欢,2.547850,-0.328366,-0.428243,0.907089,-0.795478,-0.067616
的,-3.976789,-3.061469,-3.696277,-2.028835,-0.797414,3.656608
水果,-0.301560,-7.355184,1.624798,-6.490716,0.959894,-3.321026
是,-5.490034,0.714860,7.482478,-0.167447,5.911946,1.401462
橙子,1.193535,-2.350952,3.166513,0.128357,7.557112,2.883754
和,1.222828,-7.410800,-4.234698,0.031570,-6.022178,3.869758
苹果,1.437257,0.422291,3.915477,-5.110336,-1.509519,-5.897037


In [14]:
K = X.T @ Wk
V = X.T @ Wv

### 相似矩阵

In [15]:
Q @ K.T

,我,喜欢,的,水果,是,橙子,和,苹果
我,-44.625524,-35.043298,-109.772883,45.209793,8.880179,24.951700,108.602293,59.083677
喜欢,15.522986,-5.836461,-7.314718,20.237246,8.660156,11.034192,20.786434,5.257586
的,-40.928513,-16.946257,-26.682033,-26.157392,-6.362524,15.673708,-39.434463,0.101797
水果,-107.650253,-30.223076,-106.784981,21.926217,31.025282,24.423659,113.163625,57.117776
是,-87.349856,23.365863,33.623299,-58.528286,3.967213,-62.265364,-23.177543,2.280620
橙子,-71.270085,0.522120,14.800393,7.783341,32.268627,-46.950580,64.970941,36.797052
和,-43.193557,-50.301811,-112.474375,13.738032,20.091349,71.952396,15.160104,25.399476
苹果,-9.245707,-2.767158,-14.005077,24.428112,22.808250,28.847424,52.318769,3.501544


### 点积缩放

In [16]:
df_QK = Q @ K.T / np.sqrt(d_k)

### SoftMax

In [17]:
df_QK.iloc[0]

我    -18.218294
喜欢   -14.306367
的    -44.814592
水果    18.456821
是      3.625318
橙子    10.186489
和     44.336701
苹果    24.120810
Name: 我, dtype: float64

In [18]:
for i in range(len(df_QK)):
    exp_v = np.exp(df_QK.iloc[i])
    softmax = exp_v / np.sum(exp_v)
    df_QK.iloc[i] = softmax

df_QK

,我,喜欢,的,水果,是,橙子,和,苹果
我,6.803166e-28,3.401258e-26,1.914633e-39,5.761173e-12,2.085793e-18,1.474862e-15,1.000000e+00,1.660926e-09
喜欢,6.001344e-02,9.799841e-06,5.359498e-06,4.112318e-01,3.643083e-03,9.602527e-03,5.145857e-01,9.082247e-04
的,9.196454e-11,1.643138e-06,3.086889e-08,3.824194e-08,1.236430e-04,9.981437e-01,1.692449e-10,1.730986e-03
水果,7.074633e-40,3.780427e-26,1.007202e-39,6.662075e-17,2.734472e-15,1.846740e-16,1.000000e+00,1.156304e-10
是,3.506827e-22,1.495577e-02,9.850361e-01,4.518331e-17,5.437868e-06,9.826393e-18,8.368815e-11,2.731466e-06
橙子,6.989786e-25,3.743005e-12,1.272793e-09,7.254954e-11,1.591752e-06,1.433216e-20,9.999883e-01,1.011032e-05
和,3.842598e-21,2.110239e-22,2.000494e-33,4.770836e-11,6.383186e-10,1.000000e+00,8.525657e-11,5.574017e-09
苹果,1.215017e-11,1.710917e-10,1.740789e-12,1.134879e-05,5.857984e-06,6.894356e-05,9.999138e-01,2.211412e-09


### V矩阵

In [19]:
V

,0,1,2,3,4,5
我,3.150210,-1.297353,-0.641361,7.996340,-0.713165,0.293019
喜欢,3.015803,1.742457,-0.213276,-2.229211,-2.169468,-0.590062
的,10.158378,1.303238,2.825769,-0.989661,-4.809810,-1.173599
水果,-7.661562,-0.557794,-9.260847,-1.664928,4.204678,3.270825
是,-5.007266,3.597308,-1.193003,-8.183287,-2.836090,1.486614
橙子,-0.246580,-0.872173,-9.193241,3.202000,2.358355,-4.815078
和,-5.112422,-4.287823,-1.883311,-6.735523,-4.491437,0.953789
苹果,-10.737088,0.803196,-6.601659,-2.885820,6.625271,5.398026


In [20]:
attention = df_QK @ V
attention

,0,1,2,3,4,5
我,-5.112422,-4.287823,-1.883311,-6.735523,-4.491437,0.953789
喜欢,-5.622680,-2.508210,-4.914578,-3.672501,-0.606647,1.817528
的,-0.265322,-0.868715,-9.187750,3.190045,2.365091,-4.796613
水果,-5.112422,-4.287823,-1.883311,-6.735523,-4.491437,0.953789
是,10.051416,1.309818,2.780270,-1.008244,-4.770279,-1.164839
橙子,-5.112479,-4.287759,-1.883358,-6.735487,-4.491322,0.953835
和,-0.246580,-0.872173,-9.193241,3.201999,2.358355,-4.815078
苹果,-5.112115,-4.287499,-1.883895,-6.734789,-4.490856,0.953420


### 位置编码

In [23]:
X.T

,0,1,2,3,4,5,6,7,8,9
我,1.468649,-0.723157,-1.293168,0.249329,2.329141,1.752795,-0.255968,0.827190,-0.514475,-1.056280
喜欢,0.512332,0.692792,0.007413,1.460608,-0.032326,-0.028772,-0.294977,0.314051,-0.700304,0.700161
的,3.384863,2.576392,-1.383175,1.751011,-2.077149,2.153421,-0.796146,-0.262360,-0.190189,0.559826
水果,0.934355,-2.026781,-0.790589,-1.110696,1.063843,-0.628010,3.817258,-1.049676,-0.557369,-0.928587
是,0.667819,0.384005,0.052669,-0.382960,-1.640072,-1.514602,1.938287,-1.456138,0.111411,-2.909087
橙子,1.005231,-1.713287,-2.331398,-0.069374,1.980515,-2.435055,0.454573,-2.107287,-0.501081,-1.240167
和,0.915805,-1.751494,1.843134,-0.005472,-0.945925,-0.718926,1.925642,3.552628,2.092016,-0.643384
苹果,-1.193919,-0.804299,0.572513,-1.500060,0.772428,-1.325299,3.144667,-0.644317,-0.995969,0.586208


In [24]:
pos, d_model = 8, 10
pos_encoding = np.zeros((pos, d_model))
pos_encoding

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [26]:
# 获取位置索引，并扩展维度进行计算
print(np.arange(pos))
position = np.arange(pos)[:, np.newaxis]
print(position)

[0 1 2 3 4 5 6 7]
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]]


In [27]:
# 计算分母
div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
print(div_term)

[1.00000000e+00 1.58489319e-01 2.51188643e-02 3.98107171e-03
 6.30957344e-04]


In [28]:
# 将正弦应用于偶数索引
pos_encoding[:, 0::2] = np.sin(position * div_term)
# 将余弦应用于奇数索引
pos_encoding[:, 1::2] = np.cos(position * div_term)
pos_encoding

array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 8.41470985e-01,  5.40302306e-01,  1.57826640e-01,
         9.87466836e-01,  2.51162229e-02,  9.99684538e-01,
         3.98106119e-03,  9.99992076e-01,  6.30957303e-04,
         9.99999801e-01],
       [ 9.09297427e-01, -4.16146837e-01,  3.11697146e-01,
         9.50181503e-01,  5.02165994e-02,  9.98738351e-01,
         7.96205928e-03,  9.99968302e-01,  1.26191435e-03,
         9.99999204e-01],
       [ 1.41120008e-01, -9.89992497e-01,  4.57754548e-01,
         8.89078609e-01,  7.52852930e-02,  9.97162035e-01,
         1.19429312e-02,  9.99928681e-01,  1.89287090e-03,
         9.99998209e-01],
       [-7.56802495e-01, -6.53643621e-01,  5.92337725e-01,
         8.05689779e-01,  1.00306487e-01,  9.94956586e-01,
         1.59236138e-02,  9.99873211e-01,  2.52382670e-03,
         9.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def positionl_encoding(pos, d_model):
    """
    pos: int, 最大长度
    d_model: int, 词向量维度
    return: np.array, 位置编码矩阵
    """
    pos_encoding = np.zeros((pos, d_model))
    position = np.arange(pos)[:, np.newaxis]
    # 分母
    div_term = np.exp(np.arrange(0, d_model,2) * -(np.log(10000.0) / d_model))
    # 偶数
    pos_encoding[:, 0::2] = np.sin(position * div_term)
    # 奇数
    pos_encoding[:, 1::2] = np.cos(position * div_term)
    return pos_encoding